In [ ]:
%load_ext autoreload
%autoreload 2

import dotenv
import json
import langchain
from pydantic.dataclasses import dataclass, Field

import interlab
from interlab import actor, context
from interlab.context import Context, with_context

import openai
import os
dotenv.load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

import time

from interlab.lang_models import WebConsoleModel
from interlab.actor import WebConsoleActor

import enum
#pip3 install aenum
from aenum import extend_enum

In [ ]:
#context browser
storage = context.FileStorage("logs") # Directory for storing contexts (structured logs)
storage.live_display(height=500)
# Alternatively, you can use storage.start_server() if you want to only open the storage in another browser tab

# Double-crux bot functions

ChatGPT4 will simulate 2 people and 1 LLM

There are alternating turns:
    Even turns: The chat bot sends out a message and selects the next speaker.
    Odd turns: JSON referenced so the correct participant replies.

In [ ]:
#import the prompts for the bot
import csv

bot_instructions = {}
with open("bot_instructions.csv", encoding='utf-8-sig') as csvfile:
    dict_reader = csv.DictReader(csvfile, escapechar='\\')
    
    # Initialize bot_instructions with keys from the first row
    bot_instructions = {key: [] for key in dict_reader.fieldnames}

    # Iterate through the rest of the file
    for row in dict_reader:
        for key, value in row.items():
            if value:
                bot_instructions[key].append(value)

#the major conversation steps that can be taken, which are constructed based on the bot prompts
class ConversationStep(enum.Enum):
    pass

for key in bot_instructions.keys():
    extend_enum(ConversationStep, key, key)

In [ ]:
#the counter increases every time the bot speaks
def act(speaker, message=None, expected_type=None):
    for i in range(3):
        try:
            return speaker.act(message, expected_type=expected_type).data
        except:
            time.sleep(30)
        print('RATE LIMIT TIMEOUT')


def conversation(bot: actor.ActorWithMemory, pa: actor.ActorWithMemory, pb: actor.ActorWithMemory, max_turns = 30):
    #default
    result = "TIMEOUT"
    
    chat_history = []
    
    step, substep = 'Start', 0
        
    for i in range(1, max_turns + 1):
        if i%3 == 1:
            speaker = bot
            message = bot_instructions.get(step)[substep].format(participant_a = pa.name, participant_b = pb.name)
            expected_action = ChatbotAction
            
        else:
            if next_turn == pa.name:
                speaker = pa
            elif next_turn == pb.name:
                speaker = pb
            
            message = f'How should you reply to the double crux facilitator chatbot?'
            expected_action = PersonAction
            
            substep += 1
        
        action = act(
                speaker,
                message = message,
                expected_type = expected_action)
        assert isinstance(action, expected_action)
        
        if speaker == bot:
            hidden_thoughts = str(action.analysis)
            #you can toggle observations of hidden thoughts to conserve tokens
            #bot.observe(f'Conversation analysis: {hidden_thoughts}')
            next_turn = str(action.next_turn.value)
    
        reply = action.reply
        chat_history.append(
            f'{speaker.name}: {reply}'
        )
        
        bot.observe(chat_history[-1:])
        pa.observe(chat_history[-1:])
        pb.observe(chat_history[-1:])
        
        if substep == len(bot_instructions.get(step)):
            if step == 'Close discussion':
                break
            action = act(
                bot,
                message = None,
                expected_type=DiscussionFlow)
            assert isinstance(action, DiscussionFlow)
            step, substep = str(action.conversation_step.value), 0
        
    return result, chat_history

In [ ]:
#human actors setup
e4 = langchain.chat_models.ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 1.2)

pa_initial_instructions = 'Your name is Clara. You believe in veganism.'
pb_initial_instructions = 'Your name is Danny. You are an omnivore.'

pa = actor.OneShotLLMActor("Clara", e4, f'{pa_initial_instructions}')
pb = actor.OneShotLLMActor("Danny", e4, f'{pb_initial_instructions}')

#bot actor setup
e4_bot = langchain.chat_models.ChatOpenAI(model_name="gpt-4", temperature = 0.7)

initial_instructions = f'You are a discussion facilitator using the applied rationality method Double Crux to help participants find \'cruxes\' upon which a disagreement hinges. \
A crux for an individual is a fact that would change their conclusion in the overall disagreement. \
A double-crux is two symmetric, opposite cruxes: One crux is a fact that, if true, disproves one participant\'s belief. The second crux is the same fact that, if false, disproves the other participant\'s belief. \
Here is the premise: {pa.name} and {pb.name} must hold opposite beliefs. \
Your job is to discover a crux such that if {pa.name} believed the crux was true (or false), then they would change their mind about their belief. \
Conversely, the opposite conclusion about the crux would change {pb.name}\'s mind. \
These cruxes would constitute a double-crux. \
The cruxes should be more concrete, falsifiable, well-defined, and/or discoverable than the beliefs. \
Double-Crux differs from typical debates which are usually adversarial, and instead attempt to be a collaborative attempt to uncover the true underlying structure of the disagreement. \n\
Address and ask questions to only one participant at a time.'                            

bot = actor.OneShotLLMActor("Double Crux Facilitator", e4_bot, f'{initial_instructions}')

#define the decisions that can be taken each turn
class TurnID(enum.Enum):
    participant_a = pa.name
    participant_b = pb.name

@dataclass
class ChatbotAction:
    analysis: str = Field(
        description="My hidden thoughts and reasoning, visible only to me.")
    next_turn: TurnID = Field(
        description="The single person who should reply next in the conversation. Switch between participants as often as possible.")
    reply: str = Field(
        description="My response, directed to the single participant who I chose to speak next.")

@dataclass
class PersonAction:
    reply: str = Field()
    
@dataclass
class DiscussionFlow:
    conversation_step: ConversationStep = Field(
        description="The next phase that this conversation should enter. \
        There are 3 choices:\n \
        Find crux = Try to find one of the participant's cruxes. You should do this at least once in every conversation, or multiple times until a crux is found that leads to a double crux.\n \
        Verify double crux = Check if the other participant shares a symmetric, opposite crux. You should do this after finding one of the participant's cruxes to see if it is a double crux.\n \
        Close discussion = End this conversation if you found a double crux, no double crux was found after a few tries, or if the conversation is out of control.\n")

with Context(f"double-crux", storage=storage) as c:
    result, history = conversation(bot, pa, pb)
    c.set_result(result)
    print("Done")
    for item in history:
        print(item)